# Video Highlights Generation

This notebook generates video highlights from a video file using Azure AI services.

## Pre-requisites
1. Install required packages

In [ ]:
import subprocess
import sys
import importlib.util

# Complete package list for AutoHighlight
packages = [
    # Core dependencies
    'openai>=1.3.3', 
    'python-dotenv',
    'requests',
    'jsonschema',
    
    # Azure AI services
    'azure-ai-vision-imageanalysis==1.0.0b3',  # Correct Azure Vision package
    'azure-identity',
    'azure-core',
    
    # Video processing (FFmpeg-based)
    'ffmpeg-python',
    'Pillow>=10.0.0',
    
    # Scientific computing
    'numpy>=1.23.5',
    'scipy>=1.10',
    
    # Optional: Audio processing (if needed for advanced features)
    'librosa==0.10.1.post2',
    'soundfile>=0.12.1',
]

def check_package_installed(package_name):
    """Check if a package is already installed"""
    spec_name = package_name.split('>=')[0].split('==')[0].split('<')[0]
    return importlib.util.find_spec(spec_name) is not None

def install_package(package):
    """Install a single package with error handling"""
    try:
        # Check if already installed
        package_name = package.split('>=')[0].split('==')[0].split('<')[0]
        if check_package_installed(package_name):
            print(f"✅ {package_name} already installed")
            return True
            
        # Install the package
        print(f"📦 Installing {package}...")
        result = subprocess.run([sys.executable, '-m', 'pip', 'install', package], 
                              capture_output=True, text=True)
        if result.returncode == 0:
            print(f"✅ Successfully installed {package}")
            return True
        else:
            print(f"❌ Failed to install {package}: {result.stderr}")
            return False
    except Exception as e:
        print(f"❌ Error installing {package}: {e}")
        return False

print("🚀 Installing AutoHighlight dependencies...")
print("=" * 50)

# Install packages
failed_packages = []
for package in packages:
    if not install_package(package):
        failed_packages.append(package)

print("\n" + "=" * 50)
print("📋 Installation Summary:")
print(f"✅ Successfully installed: {len(packages) - len(failed_packages)}/{len(packages)} packages")

if failed_packages:
    print(f"❌ Failed packages: {len(failed_packages)}")
    for pkg in failed_packages:
        print(f"   - {pkg}")
    print("\n💡 You may need to install these manually or check your internet connection")
else:
    print("🎉 All packages installed successfully!")

# Check critical dependencies
print("\n🔍 Verifying critical imports...")
critical_imports = [
    ('openai', 'OpenAI API client'),
    ('azure.ai.vision.imageanalysis', 'Azure AI Vision'),
    ('ffmpeg', 'FFmpeg Python wrapper'),
    ('numpy', 'NumPy for numerical computing'),
    ('PIL', 'Pillow for image processing')
]

import_results = []
for module, description in critical_imports:
    try:
        __import__(module)
        print(f"✅ {description}")
        import_results.append(True)
    except ImportError:
        print(f"❌ {description} - Import failed")
        import_results.append(False)

# FFmpeg system check
print("\n🎬 Checking FFmpeg availability...")
try:
    result = subprocess.run(['ffmpeg', '-version'], capture_output=True, text=True, timeout=5)
    if result.returncode == 0:
        version_line = result.stdout.split('\n')[0]
        print(f"✅ FFmpeg found: {version_line}")
    else:
        print("❌ FFmpeg command failed")
        print("💡 Please install FFmpeg: https://ffmpeg.org/download.html")
except (FileNotFoundError, subprocess.TimeoutExpired):
    print("❌ FFmpeg not found in system PATH")
    print("💡 Install FFmpeg:")
    print("   - Windows: winget install ffmpeg")
    print("   - Or download from: https://ffmpeg.org/download.html")

print(f"\n🎯 Pipeline readiness: {'READY' if all(import_results) else 'NEEDS ATTENTION'}")
print("Package installation complete!")

## API Configuration

**New users:** Before running the pipeline, you need to configure your Azure API credentials.

In [ ]:
# Azure AI Services (for video analysis)
AZURE_AI_ENDPOINT = ""

# Azure OpenAI (for highlight reasoning)
AZURE_OPENAI_ENDPOINT = ""
AZURE_OPENAI_API_KEY = ""

# Validate that credentials have been updated
if "your-ai-service" in AZURE_AI_ENDPOINT:
    raise ValueError("⚠️ Please update AZURE_AI_ENDPOINT with your actual Azure AI service endpoint!")

if "your-openai-service" in AZURE_OPENAI_ENDPOINT:
    raise ValueError("⚠️ Please update AZURE_OPENAI_ENDPOINT with your actual Azure OpenAI endpoint!")

if AZURE_OPENAI_API_KEY == "your_openai_api_key_here":
    raise ValueError("⚠️ Please update AZURE_OPENAI_API_KEY with your actual API key!")

# Set environment variables for other modules to use
import os
os.environ["AZURE_AI_ENDPOINT"] = AZURE_AI_ENDPOINT
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT  
os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY

print("✅ API credentials configured successfully!")
print(f"Azure AI Endpoint: {AZURE_AI_ENDPOINT}")
print(f"Azure OpenAI Endpoint: {AZURE_OPENAI_ENDPOINT}")
print(f"API Key: {AZURE_OPENAI_API_KEY[:8]}...{AZURE_OPENAI_API_KEY[-4:]}")

## Pipeline Configuration

**What this does:** Configure the AI pipeline to create highlights tailored to your video content and preferences.

**Key settings you control:**
- 🎥 **Video source** - Path to your video file
- 🎬 **Content type** - What kind of video you're analyzing (sports, keynote, etc.)
- ⏱️ **Target length** - How long you want your final highlight video
- 🎯 **Style preferences** - Clip density, transitions, effects
- 🚀 **Personalization** - Anything specific you expect your highlights to pertain to

**Important:** You must specify the `SOURCE_VIDEO_PATH` and choose the right `VIDEO_TYPE` before running the notebook. The VIDEO_TYPE determines what events the AI looks for (e.g., "soccer" finds goals and exciting plays, "keynote" finds key quotes and audience reactions).

In [ ]:
# 🎥 VIDEO CONFIGURATION

import time
# Replace with your actual video file path
SOURCE_VIDEO_PATH = r""  # e.g., r"C:\Users\YourName\Videos\video.mp4"

# 📁 OUTPUT DIRECTORY (will use the same directory as your video file by default)
OUTPUT_DIR = os.path.dirname(os.path.abspath(SOURCE_VIDEO_PATH if SOURCE_VIDEO_PATH else __file__))

# 🎬 SCHEMA CONFIGURATION (schema types available in ./schemas/ folder)
VIDEO_TYPE = "soccer"  # Options: "soccer", "basketball", "football", etc.

# Path to the schema file (this will be auto-configured based on VIDEO_TYPE)
SCHEMA_PATH = None  # This will be set automatically

# 🎯 HIGHLIGHT GENERATION PARAMETERS
TARGET_DURATION_S = 60          # Target duration for the final highlight video in seconds
CLIP_DENSITY = "medium"          # Options: "low", "medium", "high"
PERSONALIZATION = "none"      # Any specific expectations from the highlight
TRANSITION_TYPE = "cut"          # Options: "cut", "fade"
SPEED_RAMP = False              # Whether to apply speed ramping
ADD_CAPTIONS = False            # Whether to add captions (requires additional setup, keep FALSE)
RESOLUTION = 720                # Output resolution height (720p, 1080p)
HUMAN_IN_THE_LOOP_REVIEW = False # Whether to pause for human review of analyzer generation

# 🚀 ADVANCED CONFIGURATION
ANALYZER_ID = f"highlight-analyzer-{int(time.time())}"  # Unique analyzer ID

# Validate configuration
if not SOURCE_VIDEO_PATH or SOURCE_VIDEO_PATH == "path/to/your/video.mp4":
    raise ValueError("⚠️ Please update SOURCE_VIDEO_PATH with your actual video file path!")

if not os.path.exists(SOURCE_VIDEO_PATH):
    raise ValueError(f"❌ Video file not found: {SOURCE_VIDEO_PATH}")

print(f"✅ Configuration validated!")
print(f"Source Video: {SOURCE_VIDEO_PATH}")
print(f"Output Directory: {OUTPUT_DIR}")
print(f"Video Type: {VIDEO_TYPE}")
print(f"Target Duration: {TARGET_DURATION_S}s")

## 1. Generate and Activate Schema

**What this does:** Creates a custom Azure AI analyzer that uses OpenAI reasoning to understand your specific video content and identify highlight-worthy moments based on your preferences.

The schema generation process takes your `VIDEO_TYPE`, `CLIP_DENSITY`, `TARGET_DURATION_S`, and `PERSONALIZATION` settings and builds an intelligent analyzer that knows exactly what to look for in your video (e.g., goals in soccer, key quotes in presentations, exciting moments in gameplay).

In [ ]:
import schema_manager

try:
    SCHEMA_PATH = schema_manager.activate_schema(
        VIDEO_TYPE, 
        CLIP_DENSITY,
        TARGET_DURATION_S,
        PERSONALIZATION,
        human_in_the_loop_review=HUMAN_IN_THE_LOOP_REVIEW
    )
    print(f"Schema activated: {SCHEMA_PATH}")
    
    # Display schema content
    with open(SCHEMA_PATH, 'r') as f:
        import json
        print(json.dumps(json.load(f), indent=2))

except Exception as e:
    print(f"Error activating schema: {e}")

## 2. Analyze Video

Now, we submit the video to Azure Content Understanding (CU) for analysis using the custom schema we just generated. This step can take a long time depending on the length of the video.

In [ ]:
# Import the analysis module
from run_video_analysis import run_analysis

# Get Azure AI endpoint from environment
AZURE_AI_ENDPOINT = os.getenv("AZURE_AI_ENDPOINT")
if not AZURE_AI_ENDPOINT:
    raise ValueError("⚠️ AZURE_AI_ENDPOINT environment variable not set! Please check your .env file.")

print("🎬 Starting video analysis...")
print(f"Video: {os.path.basename(SOURCE_VIDEO_PATH)}")
print(f"Schema: {os.path.basename(SCHEMA_PATH)}")

# Run the analysis with all parameters passed explicitly
success, ANALYSIS_RESULT_PATH, error = run_analysis(
    video_path=SOURCE_VIDEO_PATH,
    schema_path=SCHEMA_PATH,
    azure_ai_endpoint=AZURE_AI_ENDPOINT,
    output_dir=OUTPUT_DIR,
    analyzer_id=ANALYZER_ID
)

if not success:
    raise Exception(f"❌ Video analysis failed: {error}")

print(f"✅ Video analysis completed successfully!")
print(f"📄 Results saved to: {ANALYSIS_RESULT_PATH}")

## 3. Parse and Pre-filter Segments

Now we receive the output from Azure CU, which describes all the video frames and their content. We apply rule-based filtering to remove frames that are not worthy enough to be part of the highlights, based on the schema's scoring criteria. This creates a simplified list of potential highlight clips.

In [ ]:
import json_parser
import json

try:
    PREFILTERED_SEGMENTS_PATH, _, _ = json_parser.process_json(
        input_path=ANALYSIS_RESULT_PATH
    )
    print(f"Segments parsed and pre-filtered. Results saved to: {PREFILTERED_SEGMENTS_PATH}")

    # Display the pre-filtered segments
    with open(PREFILTERED_SEGMENTS_PATH, 'r') as f:
        segments_data = json.load(f)
        print(json.dumps(segments_data, indent=2))

except Exception as e:
    print(f"Error parsing segments: {e}")

## 4. Highlight Edits Filtering

Now we send the parsed output to OpenAI o1 for advanced reasoning and intelligent timestamp selection. The AI analyzes all the potential highlight clips and selects the specific timestamps that will create the most compelling custom highlights catered to your preferences, meeting the `TARGET_DURATION_S` and arranging them in an optimal narrative order.

In [ ]:
import build_highlight
import json

try:
    # Set the global variables that build_highlight.main() expects
    build_highlight.SEGMENTS_PATH = PREFILTERED_SEGMENTS_PATH
    build_highlight.RUNTIME_S = TARGET_DURATION_S
    
    # Call main function (no parameters needed)
    build_highlight.main()
    
    # The output file is hardcoded to "final_highlight_result.json" in the same directory
    HIGHLIGHT_PLAN_PATH = "final_highlight_result.json"
    print(f"Highlight plan created. Saved to: {HIGHLIGHT_PLAN_PATH}")

    # Display the highlight plan
    with open(HIGHLIGHT_PLAN_PATH, 'r') as f:
        plan_data = json.load(f)
        print(json.dumps(plan_data, indent=2))

except Exception as e:
    print(f"Error building highlight plan: {e}")

## 5. Stitch Video Clips

From the feedback we received from OpenAI's reasoning, we now work on stitching together the custom user highlights. This final step takes the intelligently selected timestamps and creates a seamless highlight video tailored to your preferences using FFmpeg for video editing.

In [ ]:
# Video Stitching using FFmpeg
import video_stitching_ffmpeg
import os

# Define output path for the highlight video
OUTPUT_HIGHLIGHT_PATH = os.path.join(OUTPUT_DIR, "highlight.mp4")

print("🎬 Running video stitching with FFmpeg...")
print(f"📄 Input video: {SOURCE_VIDEO_PATH}")
print(f"📄 Highlight plan: {HIGHLIGHT_PLAN_PATH}")
print(f"📄 Output video: {OUTPUT_HIGHLIGHT_PATH}")

try:
    result = video_stitching_ffmpeg.stitch_video(
        video_path=SOURCE_VIDEO_PATH,
        plan_path=HIGHLIGHT_PLAN_PATH,
        output_path=OUTPUT_HIGHLIGHT_PATH,
        transition=TRANSITION_TYPE,
        speed_ramp=SPEED_RAMP,
        resolution=RESOLUTION
    )
    
    if result:
        print(f"✅ SUCCESS: Highlight video created at {result}")
        
        # Check file info
        if os.path.exists(result):
            size_mb = os.path.getsize(result) / (1024*1024)
            print(f"   📁 File size: {size_mb:.2f} MB")
            print(f"   📁 Full path: {result}")
        print(f"\n🎉 Highlight generation pipeline completed successfully!")
    else:
        print("❌ Video stitching failed")
        
except Exception as e:
    print(f"❌ ERROR during video stitching: {e}")
    import traceback
    traceback.print_exc()